# Advanced Fisheries Economics workshop

## Empirical bio-economic modelling


**Aim**:  Estimate the unknown parameters in the Gordon-Schaefer fisheries model using data from the fishery

What we need:

- Data (There is lots of data but data infrastrcuture and data governance is not well developed in the Pacific. Lack of data is not the problem) In recent work we have been working with over 970,000 observations so we are approaching data sets of 1 million observations in some of our work.

- A way to manipulate data (Pandas)

- A way to estimate paprameters (Statsmodels)

- A way to extract the policy from the parameter estimates or relate the policy to the parameter estimates

This is an example of structural econometrics.

## Where do we get data?


- Market and trade data: Company websites (thai Union) and government trade statistics (EU, Japan, Thailand, US)

- Data aggregators [Quandl](https://www.quandl.com/) has an API and [directly accessible using Python](https://www.quandl.com/tools/python)

- Central Banks US Federal Reserve [FRED](https://fred.stlouisfed.org/) database ([API](https://github.com/mortada/fredapi)), others by downloading

- SPC (usually sent via Slack in spreadsheet form very inefficient

- Members (usually e-mailed in a spreadsheet also very inefficient) 

- API (Application programming interface) World Bank, UN Comtrade, Quandl

- Webscraping company websites - Also a possible workshop topic

- Surveys (last reort) - This might be a topic of a futture workshop on survey sampling methods and design



## Where do we put data?


- relational database (Postgresql) in the cloud (Safe and secure)

- spreadsheets (old way of doing it): Unsafe and difficult to manage

- How do we get data from a database: **E**xtract **T**ransform **L**oad  (ETL) tool.



## Reproducibility


- Using API's enhances reproducibility (automation of data collection helps)

- Webscraping enhances reproducibility

- Reproducibility means we want to be able to reproduce graphs and results in any reports in such a way that they can be traced back to the original data. Think of it as traceability for data and report writing. 



In [4]:
#!pip install psycopg2
import psycopg2 #need this to connect to the database you will need to install this from the command line with pip install psycopg2
# This is why you need administrator rights on your computer. Alternatively load the notebook to Google colaboratory and install 
#it there using !pip install psycopg2. You have limited administrator rights on Google colaboratory.
import sqlite3
import io
from sqlalchemy import create_engine
#import gspread # To link to google sheets only needed for google colaboratory
import pandas as pd
import numpy as np

### Using PANDAS

Make sure you do the tutorial 10 minutes to Pandas

[10 Minutes to Pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html) (Do this as an **exercise!**)

Also it's worth reading the [Intro to Pandas](https://pandas.pydata.org/pandas-docs/stable/getting_started/index.html#getting-started)

We have already used Pandas a little above df is a Pandas data frame.

We can also import data from a spreadsheet directly into Pandas. This is covered in section 2 of Intro to Pandas.


A good reference for [SQL](https://www.w3schools.com/sql/)



## Connecting to a database


conn = psycopg2.connect("host=  dbname= user= password= ") establishes a connection

You need to enter the details that you will be provided with for host, dbname and password

cur = conn.cursor() 


Note this can in theory also be done using Excel Powerquery but may be blocked by your IT department.



In [ ]:
conn = psycopg2.connect("host=  dbname= user= password= ") 
cur = conn.cursor() 

## Queries


- Now we write a query in SQL (Structured Query Language - pronounced sequel by purists)

- We execute the query cur.execute

- We fetch the data cur.fetchall()

- we pass the data to a data frame (a list of vectors or columns)

- Finally check you have the data



In [ ]:
cur.execute('SELECT * FROM ace;') # write SQL query in brackets
data = cur.fetchall()
df = pd.DataFrame(data) # Need to add headers after this call. So need to document this in Manual
df #This is a data frame you can writexl to write to Excel file

#close the connection after you ahve obtained the data

cur.close() 
conn.close()

## Data cleaning with Pandas


- Extract the data you need to fit catch to effort (days)

- do this by selecting the two columns you need from the data frame df




## Plotting


First plot your data. Use a scatter plot for this. Catch on the vertical axis and effort on the horizontal.



## Econometrics

We will use the equations from the previous notebook afew-present-5.ipynb

- we wish to estimate the relationship between catch and effort the sustainable yield curve from the alst notebook

- For the Gordon-Schaefer model that is a quadratic relationship

$$ y_i = \beta_0 +\beta_{i} X_{i1} + \beta_2 X_{i}^2 + \epsilon_i, i=1,\dots,n$$

The theory tells us that $\beta_0 = 0$.

where $X_{i}$ is total fishing effort.

Now we ccould just estimate this relationship from the data. However, the theory tells us that $E_{MEY}$ depdns on the model parameters 



## Estimate the model


The model can be estimated using ordinary least squares (linear regression/line of best fit).



In [7]:
import statsmodels.api as sm #Import statsmodels for econometrics

# need to define y and X you should already have extracted the data and plotted it
# so you just need to pass catch to y and days and day squared to X


model = sm.OLS(y, X)
results = model.fit(cov_type='HAC',cov_kwds={'maxlags':1})
#print(results.summary())

## Standard errors


- Econometric practice has changed over time 

- "Credibility revolution"

- Use of asymptotic rather than small sample standard errors

- [Newey-West robust standard errors](https://en.wikipedia.org/wiki/Newey%E2%80%93West_estimator) 

- use of robustness tests

- Reduced form is associated with the credibility revolution

- Here we are doing a very simple structural model

## Extract the parameters


In [ ]:
params = pd.DataFrame(results.params)
hh = params[:1]
hsq = params[1:]
b0 = hh[0][0]
b1 = - hsq[0][0]

## Calculating MEY from the estimated relationship
To determine maximum economic yield we solve the following problem

$$\max \Pi (E) = p h(E)- cE $$
subject to

$$\frac{dX}{dt} = rX(1-\frac{X}{K})- h(E)= 0$$
and $Y(E) = qXE$.

In stready state (if you are fishing sustainably) the constraint can be rearranged to obtain

$$X = K (1 - \frac{q}{r} E) $$
Substituting into the total rent results in

$$ \Pi (E) = p q K (1 - \frac{q}{r} E) E - c E = p q K E - \frac{p q^2 K}{r} E^2  - c E $$
Maximizing and solving for E results in

$$ E = \frac{r(pqK - c)}{2 pq^2 K} $$
which is exactly half of open access effort level. To determine this level we need the price $p$.

Note that sustainable harvest is given by

$$ q K (1 - \frac{q}{r} E) E  =  q K E - \frac{q^2 K}{r } E^2$$

Setting $ q K = b_0 $ and $ \frac{q^2 K}{r} = b_1$. Wecan then fit a quadratic regression but this implies a constraint on the parameters that depends on the parameter values and so would have to be estimated with Bayesian methods. Alternatively, we can use non-linear least-squares to fit the model.

We can rewrite $E_{MEY}$ as

$$ \frac{r(p b_0  - c)}{2 p r b_1} = \frac{p b_0 - c}{2 p b_1}$$

 
How much will be caught at this level of fishing?

$$Y = q E X$$

where $X = K (1 - \frac{q}{r} E)$

So

$$Y_{TAC} = q E K (1 - \frac{q}{r} E) = qK E - \frac{q^2 K}{r} E^2 = b_0 E -b_1 E^2$$$$ =  b_0 \frac{p b_0 - c}{2 p b_1} -b_1 (\frac{p b_0 - c}{2 p b_1})^2 $$$$ = \frac{p b_0^2 - c b_0}{2 p b_1} - \frac{(p b_0 - c)^2}{4 p^2 b_1}$$

We can now compute this and compare with the historical TAC.



## How to calculate costs


- Costs often tobtained from vessel surveys but this requires careful planning to avoid collecting biased data

- Alternatively costs can be calculated from catch, the price of fuel/oil and fuel utilization per mt of catch and information on what proportion of the total costs are due to fuel use

- This can be validated against other data in the region.



## Surveys


- First identify a sampling frame (A list of all member sof the population, i.e. **a list of all vessels**)

- Radomly select a sample from this population

- How many to sample?

We are interested in the mean cost of fishing per day. We will assume the cost of fishing is normally distributed for purposes of illustration and that we are examining a single gear type.





Then if $\bar{x}$ is the mean cost of fishing the standard error of the cost of fishing is given by $\frac{\sigma}{\sqrt{n}}$

The sampling bias is given by $\mu - \bar{x}$

The confidence interval for $\mu$ is

$$\bar{x} - Z_{\frac{\alpha}{2}}\frac{\sigma}{\sqrt{n}} \leq \mu \leq \bar{x} + Z_{\frac{\alpha}{2}}\frac{\sigma}{\sqrt{n}} $$

If we set an error tolerance for the sampling error so that

$$ \mu -\bar{x} \leq \epsilon$$

Then $$Z_{\frac{\alpha}{2}}\frac{\sigma}{\sqrt{n}} \leq \epsilon$$

From this we obtain that $n > Z_{\frac{\alpha}{2}}^2\frac{\sigma^2}{\epsilon^2}$.

Usually we set $\alpha = 0.05$ for a 9
%% confidence interval, $\sigma$ we otain from previous studies, e.g. past surevys. $\epsilon$ we choose what error we are prepared to accespt and $Z$ we obtain from the normal distribution tables or from the computer.



In [ ]:
EMEY
print('MEY:',EMEY)
EMSY = b0/(2*b1)
p = 1500 # price 
c = 10000 # costs per day 

yTAC = ((p(b0**2) - c*b0)/(2*p*b1)) - ((p*b0 - c)**2)/(4*(p**2)*b1)
Ymsy = 0.25*(b0**2)/b1
rent = p*(b0*EMEY - b1*EMEY**2) - c*EMEY
#M = (r*(p*q*K+c)**2)/(4*K*(p*q)**2) need to express n terms of b0 and b1 can use this to compare to SAM18 M=0.4
print('MSY:',EMSY)
print('Difference:',EMSY-EMEY)
print('TAC',yTAC)
print('TAC at MSY',Ymsy) 
print('Rent',rent)
#print('Natural mortality at MEY',M
bb1 = yTAC

## Sensitivity analysis (Uncertainty quantification)

To conduct the [sensitivity analysis](https://en.wikipedia.org/wiki/Sensitivity_analysis) we will use the Sobol sensitivity analysis method, this differs somewhat from a ceteris-paribus approach to sensitivity analysis which is a local method. Sobol or [variance-based sensitivity analysis](https://en.wikipedia.org/wiki/Variance-based_sensitivity_analysis) is a global method, So that it allows us to systematically examine variations in multiple parameters, e.g. price and cost but also the biologial parameters within given confidence bounds.

In [ ]:
from SALib.sample import saltelli
from SALib.analyze import sobol 
#from SALib.test_functions import Ishigami

In [ ]:
def gordonschaefer(par):
    hh = params[:1]
    hsq = params[1:]
    b0 = hh[0][0]
    b1 = - hsq[0][0]
    #p = (155455707/46620575)*1000 # Albacore price per tonne 2018 Thailand average import price
    #p = X[0]
    
    p = param_values[i][0]
    E = np.linspace(0,500000,10000)
    y_pred = b0*E - (E**2)*b1
    rev = p*y
    rev_pred = p*y_pred
    c = 1.01 # cost per unit effort 
    
    #c = x[1]
    #c = param_values[i][1]
    
    cost = c*E
    EMEY = (p * b0 - c)/(2*p*b1)
    EMSY = b0/(2*b1)
    yTAC = ((p*(b0**2) - c*b0)/(2*p*b1)) - ((p*b0 - c)**2)/(4*(p**2)*b1)
    Ymsy = 0.25*(b0**2)/b1
    rent = p*(b0*EMEY - b1*EMEY**2) - c*EMEY
    
    #avghooksperday = df.loc[df['yy']==2017]['hooks per day'].mean() # actual hooks per day.
    #vesseldays = EMEY/avghooksperday
    #actualdays = df.loc[df['yy']==2017]['days'].sum()
    #averagedays = actualdays/n2017
    #novessels = vesseldays/averagedays # one could also calculate this by 
    # flag rather than a broad average, this would probably smooth negotiations
    return(EMEY)

    # need to take a statsmodels object as an instance and transorm parameter estimates within this function
    # Model output might be choosable, examples include vessel numbers, hooks, rent, etc.
    


# Define the problem
problem = {
    'num_vars': 2,
    'names': ['p', 'c'],
    'bounds': [[2000, 5000],
               [0.0, 200.0]]
}

param_values = saltelli.sample(problem, N = 1000)

Y = np.zeros([param_values.shape[0]])

for i, X in enumerate(param_values):
    #print(i,X)
    Y[i] = gordonschaefer(X)
    

#print(Y)

In [ ]:
si = sobol.analyze(problem, Y, calc_second_order = False,print_to_console=True) #this is producing an error
print(si)

In [ ]:
plt.hist(Y)
plt.title('Histogram of vessel numbers using Sobol sensitivity analysis 1000 model runs')
plt.xlabel('Vessel Numbers')
plt.ylabel('Frequency')

In [ ]:
res = pd.DataFrame(si)
res.rename(index={0:'price',1:'cost per unit effort'},inplace=True)
res

S1 is the ratio of the variance of vessel days conditional on a particular parameter value to the variance of vessel days. It is the first-order Sobol sensitivity index. The results indicate that cost per hook has a far greater impact on the sensitivity of the number of vessels than does the price.